In [3]:
import os
import sys
import glob
import scipy
import sklearn
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42
pd.set_option('display.max_columns', 100)

sns.set_palette("Set2")


In [4]:
data_loc = '/home/mattolm/user_data/IndianMicrobiome/tables/'
Pdb = pd.read_csv(data_loc + 'IndianMicrobiome_basicinfo_v2.csv')
Pdb


sample                                              read1  \
0      AK_SR_1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
1      AK_SR_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
2    AK_SR_2.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
3      AK_SR_3  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
4      AK_SR_4  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
5      AK_SR_5  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
6      AK_SR_6  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
7      AK_SR_7  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
8      AK_SR_8  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
9      AK_SR_9  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
10    AK_SR_10  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
11  AK_SR_10.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
12     AK_SW_1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
13     AK_SW_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
14     AK_SW_3  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
15     AK_SW_4  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
16     AK_SW_5  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
17     AK_SW_7  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
18     AK_SW_8  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
19     AK_SW_9  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
20    AK_SW_11  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
21    AK_SW_12  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
22    AK_SW_14  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
23     AK_SG_7  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
24    AK_SG_10  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
25    AK_SG_11  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
26    AK_SG_16  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
27    AK_SG_17  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
28    AK_SG_19  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
29    AK_SG_20  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
30    AK_SG_22  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
31    AK_SG_25  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
32    AK_SG_26  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
33    AK_SG_29  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
34     AK_SG_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
35     AK_SG_4  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
36     AK_SG_5  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
37     AK_SG_8  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
38   AK_SG_8.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
39     AK_SG_9  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
40    AK_SG_12  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
41    AK_SG_15  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
42    AK_SG_18  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
43    AK_SG_28  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
44    AK_SK_10  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
45    AK_SK_19  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
46    AK_SK_32  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
47    AK_SK_35  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
48    AK_SK_44  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
49     AK_SK_1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
50    AK_SK_11  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
51    AK_SK_12  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
52    AK_SK_13  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
53    AK_SK_15  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
54    AK_SK_24  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
55    AK_SK_30  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
56    AK_SK_33  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
57    

## Run mapping

In [7]:
def submit_aegea_job(cmd, expected_output, rdb=None, alocation=None, verbose=False):
    """
    Submit the cmd to aegea, capture the jobID, and store the jobID at ~/aegea_logs.txt
    
    v1.0 - 1/26/21
    
    aegea 2.6.9
    """
    from datetime import datetime
    import subprocess
    
    if alocation is None:
        alocation = '/home/mattolm/.aegea_logs.txt'
    
    #1) Check it output already exists
    if check_s3_file(result):
        if verbose:
            print(f"{result} already exists")
        return None
    
    #2) Check if job is already running
    
    # Get list of running jobs
    if rdb is None:
        rdb = load_running_aegea(verbose=False)
    running_jobs = set(rdb[rdb['job_status'].isin(['RUNNING', 'RUNNABLE', 'STARTING'])]['job_ID'].tolist())
    if verbose:
        print(f"{len(running_jobs)} jobs are running")
    
    
    # Get output -> job key
    adb = pd.read_csv(alocation, sep='\t', names=['job_ID', 'output', 'time', 'cmd'])
    cdb = adb[adb['output'] == expected_output]
    if len(cdb) > 0:
        db = cdb[cdb['job_ID'].isin(running_jobs)]
    else:
        db = pd.DataFrame()
    if verbose:
        print(f"Captured {len(adb)} aegea logs, {len(running_jobs)} running jobs, {len(cdb)} previous attempts, {len(db)} currently running attempts")
    
    if len(db) > 0:
        if verbose:
            print(f"{expected_output} is currently running (job={db['job_ID'].tolist()})")
        return None
    
    # 3) Run job
    out = subprocess.check_output(cmd, shell=True, text=True)
    ID = eval(out)['jobId']
    
    # 4) Store job ID
    now = datetime.now()
    dt_string = now.strftime("%d.%m.%Y %H:%M:%S")
    with open(alocation, 'a') as o:
        o.write(f"\n{ID}\t{expected_output}\t{dt_string}\t{cmd}")
        
    # 5) Finish
    if verbose:
        print(f"Job {ID} is launched to create {expected_output}")
    return ID

def load_running_aegea(queue="sonnenburg__spot100", tries=20, verbose=True):
    """
    Return a list of running aegea jobs
    
    v1.1 - 3/15/21
    * Better printing of status while running showq
    
    v1.0 - 1/26/21
    
    aegea 2.6.9
    """
    import subprocess
    print("Running showq...")
    while tries >= 0:
        try:
            raw_out = subprocess.check_output(f"aegea batch ls --queue {queue}", shell=True, text=True)
            break
        except:
            #print(f"showq failed for {queue}, try # {tries}")
            tries = tries - 1
    print("Showq succeeded")

    table = {'job_ID':[], 'job_status':[], 'image':[]}
    lines = len(raw_out.split('\n'))
    for j, line in enumerate(raw_out.split('\n')):
        # Skip header
        if j >= 3:
            
            lw = line.strip().split('│')
            
            # Skip weird lines
            if len(lw) != 14:
                continue
            
            table['job_ID'].append(lw[1].strip())
            table['job_status'].append(lw[4].strip())
            table['image'].append(lw[8].strip())

    db = pd.DataFrame(table)
    
    if verbose:
        print(f"{len(db[db['job_status'] == 'RUNNING'])} aegea jobs are currently running")
    
    return db
    
def make_mapping_command(r1, r2, fasta_loc, s3_store_folder, **kwargs):
    '''
    Generate an aegea Bowtie2 command
    
    Version 1.3 - 5.10.2021
    - Add SUMMARY output type
    
    Version 1.2 - 2.25.2021
    - dont add buckets to r1 and r2
    
    Version 1.1 - 10.20.2020
    - update ret_result when using an index
    
    Version 1.0 - 8.25.2020
    
    Arguments:
        bam_loc = s3 location of .bam file
        fasta_loc = s3 location of .fasta file
        s3_store_loc = s3 folder to store results
    '''
    # Get s3 arguments
    bucket_id = kwargs.get('bucket_id', "czbiohub-microbiome")
    image = kwargs.get('image', 'sonnenburglab/bowtie2:dev')
    ret_result = kwargs.get('ret_result', False)
    
    # Get aegea arguments
    ram = kwargs.get('ram', 32000)
    cores = kwargs.get('cores', 16)
    
    # Get mapping arguments
    large_index = kwargs.get('large_index', False)
    store_index = kwargs.get('store_index', False)
    index_location = kwargs.get('index_location', False)
    output_type = kwargs.get('output_type', 'BAM')
    
    # Generate the base command
    bucket = "s3://{0}/".format(bucket_id)
    cmd = f"""
    ./prepare.sh; conda activate work; ./run_bowtie2.py --r1 {r1} --r2 {r2} --results_directory {s3_store_folder} --output_type {output_type} -p {cores}
    """.replace('\n', ' ')
    
    if index_location == False:
        cmd += ' --fasta {0}'.format(fasta_loc)
        result =  s3_store_folder + "{0}-vs-{1}.sorted.bam".format(os.path.basename(fasta_loc), os.path.basename(r1))
    else:
        cmd += ' --index {0}'.format(index_location)
        result =  s3_store_folder + "{0}-vs-{1}.sorted.bam".format(os.path.basename(index_location), os.path.basename(r1))
    
    if large_index:
        cmd += ' --large_index'
        
    if store_index:
        cmd += ' --store_index'

    if output_type == 'SUMMARY':
        result = result.replace('.sorted.bam', '.txt')
        
    # Remove variable length whitespace
    cmd = ' '.join(cmd.split()) 
    
    # Wrap in aegea
    aegea_cmd = "aegea batch submit --queue sonnenburg__spot100 --image {3} --storage /mnt=500 --vcpus {0} --memory {1} --command=\"{2}\" &>> AEGEA.log".format(
        cores, ram, cmd, image)
    
    
    if ret_result:
        return aegea_cmd, result
    else:
        return aegea_cmd
    
    return aegea_cmd


In [8]:
import io
import boto3

def get_matching_s3_objects(bucket, prefix="", suffix=""):
    """
    Generate objects in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    kwargs = {'Bucket': bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix, )
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                return

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(bucket, prefix="", suffix=""):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    for obj in get_matching_s3_objects(bucket, prefix, suffix):
        yield obj["Key"]
        
def check_s3_file(floc):
    '''
    Return True if exists and False if it does not
    '''
    bucket = floc.split('/')[2]
    prefix = '/'.join(floc.split('/')[3:])
    
    found = False
    for key in get_matching_s3_keys(bucket, prefix):
        if prefix in key:
            found = True
    return found

def store_s3_file(bucket, location, binary_string):
    s3 = boto3.resource('s3')
    object = s3.Object(bucket, location)
    object.put(Body=binary_string)
    
def load_coverage_report(s3_bucket, s3_key, sep='\t', names=None):
    '''
    https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    '''
    # Load the data from s3
    client = boto3.client("s3")
    obj = client.get_object(Bucket=s3_bucket, Key=s3_key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), sep=sep, names=names)

    return df

def load_coverage_report2(s3_loc, sep='\t', names=None):
    '''
    https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    '''
    s3_bucket = s3_loc.split('/')[2]
    s3_key = '/'.join(s3_loc.split('/')[3:])
    
    # Load the data from s3
    client = boto3.client("s3")
    obj = client.get_object(Bucket=s3_bucket, Key=s3_key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), sep=sep, names=names)

    return df
    
def read_s3_file(s3_bucket, s3_key):
    s3 = boto3.resource('s3')
    obj = s3.Object(s3_bucket, s3_key)
    return obj.get()['Body'].read().decode("utf-8")

def object_size(s3_bucket, s3_key):    
    return boto3.resource('s3').Bucket(s3_bucket).Object(s3_key).content_length

def object_size2(s3_loc):
    s3_bucket = s3_loc.split('/')[2]
    s3_key = '/'.join(s3_loc.split('/')[3:])
    
    return boto3.resource('s3').Bucket(s3_bucket).Object(s3_key).content_length
    
def read_s3_file2(s3_loc):
    s3 = boto3.resource('s3')
    bucket = s3_loc.split('/')[2]
    key = '/'.join(s3_loc.split('/')[3:])
    return read_s3_file(bucket, key)

def store_s3_file2(s3_loc, binary_string):
    s3 = boto3.resource('s3')
    bucket = s3_loc.split('/')[2]
    key = '/'.join(s3_loc.split('/')[3:])
    return store_s3_file(bucket, key, binary_string)

In [15]:
rdb = load_running_aegea()
s3_store_folder = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/mapping/'
for i, row in Pdb.iterrows():
    
    cmd, result = make_mapping_command(row['read1'], row['read2'], 'junk', s3_store_folder, index_location='s3://czbiohub-microbiome/Sonnenburg_Lab/Hadza_Nepal_Metagenomics/analysis/GenomeSetDelta/GenomeSet_delta', output_type='BAM', ret_result=True)
    submit_aegea_job(cmd, result, verbose=True, rdb=rdb)
    

Running showq...
Showq succeeded
173 aegea jobs are currently running
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/mapping/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_1_R1.fastq.gz.sorted.bam already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/mapping/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_2_R1.fastq.gz.sorted.bam already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/mapping/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_2.2_R1.fastq.gz.sorted.bam already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/mapping/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_3_R1.fastq.gz.sorted.bam already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/mapping/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_4_R1.fastq.gz.sorted.bam already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMi

## Run inStrain quick_profile

In [23]:
def make_inStrain_command(bam_loc, fasta_loc, s3_store_folder, **kwargs):
    '''
    Generate an aegea inStrain command
    
    Version 1.2 - 2.18.2021
    # Add quick_profile options
    
    Version 1.1 - 2.1.2021
    # Remove bucket appending
    
    Version 1.0 - 6.3.2020
    
    Arguments:
        bam_loc = s3 location of .bam file
        fasta_loc = s3 location of .fasta file
        s3_store_loc = s3 folder to store results
    '''
    # Get s3 arguments
    image = kwargs.get('image', 'mattolm/instrain:latest')
    queue = kwargs.get('queue', 'sonnenburg__spot100')
    ret_result = kwargs.get('ret_result', False)
    
    # Get aegea arguments
    ram = kwargs.get('ram', 178000)
    cores = kwargs.get('cores', 32)
    
    # Get inStrain aruments
    stb_loc = kwargs.get('stb_loc', None)
    gene_loc = kwargs.get('gene_loc', None)
    IS_name = kwargs.get('IS_name', None)
    command = kwargs.get('command', None)
    cmd_args = kwargs.get('cmd_args', '--skip_plot_generation --database_mode')
    timeout = kwargs.get('timeout', 172800) # This is 2 days worth of seconds; make None to ignore
    
    # Parse inStrain arguments
    if IS_name is None:
        IS_name = os.path.basename(bam_loc) + '.IS'
    cmd_args += ' -p {0}'.format(cores)
    
    # Generate the base command
    cmd = """
        ./prepare.sh; conda activate work; pip install instrain --upgrade; ./run_instrain.py 
        --bam {0} 
        --fasta {1} 
        --results_directory {2} 
        --wd_name {3} 
        --cmd_args='{4}'
        """.format(bam_loc, 
                   fasta_loc,
                   s3_store_folder,
                   IS_name,
                   cmd_args).replace('\n', ' ')
    
    # Add to base command
    if timeout is not None:
        cmd += ' --timeout {0}'.format(timeout)
    if gene_loc is not None:
        cmd += ' --genes {0}'.format(gene_loc)
    if stb_loc is not None:
        cmd += ' --stb {0}'.format(stb_loc)
    if command is not None:
        cmd += '  --command {0}'.format(command)
        
    # Remove variable length whitespace
    cmd = ' '.join(cmd.split()) 
    
    # Wrap in aegea
    aegea_cmd = "aegea batch submit --queue {4} --image {3} --storage /mnt=500 --vcpus {0} --memory {1} --command=\"{2}\" &>> AEGEA.log".format(
        cores, ram, cmd, image, queue)
    
    # Get result
    result =  s3_store_folder + IS_name + '/raw_data/Rdic.pickle'.format(IS_name)
    if command == 'quick_profile':
        result =  s3_store_folder + IS_name + '/coverm_raw.tsv'.format(IS_name)
    
    if ret_result:
        return aegea_cmd, result
    else:
        return aegea_cmd

In [30]:
rdb = load_running_aegea()

stb_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/Hadza_Nepal_Metagenomics/analysis/GenomeSetDelta/GenomeSet_delta.stb'
gene_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/Hadza_Nepal_Metagenomics/analysis/GenomeSetDelta/GenomeSet_delta.genes.fna'
fasta_loc = 's3://czbiohub-microbiome/Sonnenburg_Lab/Hadza_Nepal_Metagenomics/analysis/GenomeSetDelta/GenomeSet_delta.fna'
#s3_store_folder = 's3://czbiohub-microbiome/Sonnenburg_Lab/mattolm/HouseholdTransmission/Delta_inStrain_profiles/'
s3_store_folder = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/quick_profile/'

ori_s3_store_folder = 's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/mapping/'
for i, row in Pdb.iterrows():
    
    cmd, bam_loc = make_mapping_command(row['read1'], row['read2'], 'junk', ori_s3_store_folder, index_location='s3://czbiohub-microbiome/Sonnenburg_Lab/Hadza_Nepal_Metagenomics/analysis/GenomeSetDelta/GenomeSet_delta', output_type='BAM', ret_result=True)
    
    cmd, result = make_inStrain_command(bam_loc, fasta_loc, s3_store_folder, stb_loc=stb_loc, timeout=86400, queue='sonnenburg__spot100', 
                                        ret_result=True, ram=32000, cores=8, command='quick_profile', cmd_args='', image="sonnenburglab/instrain:latest")
    
    #print(cmd)
    submit_aegea_job(cmd, result, verbose=True, rdb=rdb)
    

Running showq...
Showq succeeded
120 aegea jobs are currently running
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/quick_profile/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_1_R1.fastq.gz.sorted.bam.IS/coverm_raw.tsv already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/quick_profile/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_2_R1.fastq.gz.sorted.bam.IS/coverm_raw.tsv already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/quick_profile/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_2.2_R1.fastq.gz.sorted.bam.IS/coverm_raw.tsv already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/quick_profile/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_3_R1.fastq.gz.sorted.bam.IS/coverm_raw.tsv already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/delta_analysis/quick_profile/GenomeSet_delta-vs-NODUP_HMN_UNMAPPED_TRIM

## Load

In [60]:
pd.set_option('display.max_colwidth', None)

In [55]:
dbs = []
for f in get_matching_s3_keys('czbiohub-microbiome', 'Sonnenburg_Lab/IndianMicrobiome/delta_analysis/quick_profile/', 'genomeCoverage.csv'):
    db = load_coverage_report('czbiohub-microbiome', f, sep=',')
    db['r1'] = f.split('/')[4].split('-vs-')[1]
    db['IS_location'] = "s3://czbiohub-microbiome/" +f
    dbs.append(db)

Gdb = pd.concat(dbs).reset_index(drop=True)
Gdb


genome   length   breadth  \
0                                    GUT_GENOME000001.fna  3221717  0.000057   
1                                    GUT_GENOME000010.fna  3707250  0.006564   
2                                    GUT_GENOME000017.fna  3932530  0.000080   
3                                    GUT_GENOME000020.fna  2823639  0.000069   
4                                    GUT_GENOME000024.fna  2221226  0.000263   
...                                                   ...      ...       ...   
346715  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  1911315  0.000229   
346716  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  2018403  0.009218   
346717  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  5186897  0.000025   
346718  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  2746266  0.000157   
346719  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  1734394  0.001236   

        coverage  reads                                                 r1  \
0       0.000091      2  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
1       0.014668    384  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
2       0.000133      4  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
3       0.000104      2  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
4       0.000647     11  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
...          ...    ...                                                ...   
346715  0.000307      4  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   
346716  0.017025    243  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   
346717  0.000050      2  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   
346718  0.000620     12  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   
346719  0.002701     34  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   

                                              IS_location  
0       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
1       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
2       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
3       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
4       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
...                                                   ...  
346715  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
346716  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
346717  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
346718  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  
346719  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  

[346720 rows x 7 columns]

## Add taxonomy information

In [58]:
GIdb.head()

genome      study substudy  \
0  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  HadzaPlus    HADZA   
1  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  HadzaPlus    NEPAL   
2  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  HadzaPlus    HADZA   
3  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  HadzaPlus    HADZA   
4  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  HadzaPlus    NEPAL   

  v1_cluster  v1_representitive    v1_score  \
0     1229_1               True  100.232778   
1     1303_2              False  102.092854   
2      804_3               True  102.991290   
3      584_1               True   96.000599   
4     1236_1               True  114.611822   

                                         sample  Completeness  Contamination  \
0      Hadza_PheChl_Fiber-Hadza-Nepal_C_14_2320         90.45           1.12   
1   Nepal_MoBio_Fiber-Hadza-Nepal_D_5_THA1074JZ         93.82           0.56   
2                Hadza_MoBio_hadza-E-H_C_4_1831         90.00           0.00   
3               Hadza_MoBio_hadza-A-D_B_21_1589         96.03           2.97   
4  Nepal_MoBio_Fiber-Hadza-Nepal_B_23_THA1066YZ         98.39           0.00   

    Length     N50  N_contigs  \
0  1099677   94538         16   
1  1458118  183633         14   
2  1002363   56456         31   
3  3634562   18986        320   
4  2623758   65468         68   

                                            GTDB_tax  \
0  d__Bacteria;p__Firmicutes;c__Bacilli;o__RF39;f...   
1  d__Bacteria;p__Firmicutes;c__Bacilli;o__RF39;f...   
2  d__Bacteria;p__Firmicutes;c__Bacilli;o__RF39;f...   
3  d__Bacteria;p__Firmicutes_A;c__Clostridia;o__C...   
4  d__Bacteria;p__Firmicutes_A;c__Clostridia;o__C...   

                                                path v2_cluster    v2_score  \
0  /LAB_DATA/CURRENT/CURRENT_Metagenomics_PROJECT...     2741_1   93.379068   
1  /LAB_DATA/CURRENT/CURRENT_Metagenomics_PROJECT...     2846_1   99.815768   
2  /LAB_DATA/CURRENT/CURRENT_Metagenomics_PROJECT...     1887_2   98.376880   
3  /LAB_DATA/CURRENT/CURRENT_Metagenomics_PROJECT...     1715_1   89.879669   
4  /LAB_DATA/CURRENT/CURRENT_Metagenomics_PROJECT...     2445_1  107.216938   

   v2_representitive v3_cluster    v3_score  v3_representitive  \
0               True     1196_0   93.379068               True   
1               True     1159_2   99.815768               True   
2               True     1167_1   98.376880               True   
3               True      959_0   89.879669               True   
4               True      968_0  107.216938               True   

                                            tRep_tax  Hadza_in_cluster  \
0  Bacteria|Firmicutes|unk|unk|unk|unk|Firmicutes...              True   
1  Bacteria|Firmicutes|unk|unk|unk|unk|Firmicutes...              True   
2            Bacteria|Firmicutes|unk|unk|unk|unk|unk              True   
3  Bacteria|Firmicutes|Clostridia|Clostridiales|C...              True   
4  Bacteria|Firmicutes|Clostridia|Clostridiales|C...              True   

   UHGG_in_cluster                                   cluster_tRep_tax  \
0             True  Bacteria|Firmicutes|unk|unk|unk|unk|Firmicutes...   
1             True  Bacteria|Firmicutes|unk|unk|unk|unk|Firmicutes...   
2            False            Bacteria|Firmicutes|unk|unk|unk|unk|unk   
3            False  Bacteria|Firmicutes|Clostridia|Clostridiales|C...   
4             True  Bacteria|Firmicutes|Clostridia|Clostridiales|C...   

   UHGG_novel_species                GTDB_species  GTDB_novel_species  \
0               False      s__CAG-884 sp000433875               False   
1               False      s__CAG-433 sp000433675               False   
2                True                         s__                True   
3                True                         s__                True   
4               False  s__Clostridium sp900540255               False   

                   NCBI_species  NCBI_novel_species  
0  Firmicutes bacterium CAG:884         

In [56]:
GIdb = pd.read_csv('/LAB_DATA/CURRENT/CURRENT_Metagenomics_PROJECTS/2019_Hadza_Nepal_Metagenomics/191119_FINAL_PROJECT_DATA/ANALYSIS/inStrain/genome_databases/Delta_10062020/GenomeSetDelta_representitiveGenomes_v2.csv')
GGdb = pd.merge(Gdb, GIdb[['genome', 'GTDB_tax', 'UHGG_in_cluster', 'GTDB_novel_species']], how='left', on='genome')
GGdb


genome   length   breadth  \
0                                    GUT_GENOME000001.fna  3221717  0.000057   
1                                    GUT_GENOME000010.fna  3707250  0.006564   
2                                    GUT_GENOME000017.fna  3932530  0.000080   
3                                    GUT_GENOME000020.fna  2823639  0.000069   
4                                    GUT_GENOME000024.fna  2221226  0.000263   
...                                                   ...      ...       ...   
346715  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  1911315  0.000229   
346716  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  2018403  0.009218   
346717  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  5186897  0.000025   
346718  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  2746266  0.000157   
346719  REFINED_METABAT215_TOP10_CONTIGS_1500_ASSEMBLY...  1734394  0.001236   

        coverage  reads                                                 r1  \
0       0.000091      2  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
1       0.014668    384  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
2       0.000133      4  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
3       0.000104      2  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
4       0.000647     11  NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SG_10_R1.fas...   
...          ...    ...                                                ...   
346715  0.000307      4  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   
346716  0.017025    243  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   
346717  0.000050      2  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   
346718  0.000620     12  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   
346719  0.002701     34  NODUP_HMN_UNMAPPED_TRIM_MARKED_TS8_B_R1.fastq....   

                                              IS_location  \
0       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
1       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
2       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
3       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
4       s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
...                                                   ...   
346715  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
346716  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
346717  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
346718  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
346719  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   

                                                 GTDB_tax  UHGG_in_cluster  \
0       d__Bacteria;p__Firmicutes_A;c__Clostridia;o__P...             True   
1       d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...             True   
2       d__Bacteria;p__Firmicutes_A;c__Clostridia;o__P...             True   
3       d__Bacteria;p__Firmicutes;c__Bacilli;o__Staphy...             True   
4       d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...             True   
...                                                   ...              ...   
346715  d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...            False   
346716  d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...            False   
346717  d__Bacteria;p__Verrucomicrobiota;c__Lentisphae...             True   
346718  d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...             True   
346719  d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...            False   

        GTDB_novel_species  
0                     True  
1                    False  
2                    False  
3                    False  
4                    False  
...                    ...  
346715                True  
346716                True  
346717               False  
346718               False  
346719                True  

[346720 rows x 10 columns]

## Parse a bit

In [57]:
GPdb = GGdb.copy()
GPdb['sample'] = [x.split('_MARKED_')[-1].split('_R1.fastq')[0] for x in GPdb['r1']]
del GPdb['r1']
GPdb = pd.merge(GPdb, Pdb[['sample', 'pairs']], how='left')
assert len(GPdb) == len(Gdb)

GPdb['rel_ab'] = GPdb['reads'] / (GPdb['pairs'] * 2)
GPdb.sort_values('rel_ab')
GPdb.groupby('sample')['rel_ab'].sum()


sample
AK_SG_10      0.874475
AK_SG_11      0.907211
AK_SG_12      0.851515
AK_SG_15      0.891183
AK_SG_16      0.897941
AK_SG_17      0.787426
AK_SG_18      0.911042
AK_SG_19      0.878438
AK_SG_2       0.887538
AK_SG_20      0.904780
AK_SG_22      0.893860
AK_SG_25      0.835755
AK_SG_26      0.842196
AK_SG_28      0.893436
AK_SG_29      0.837449
AK_SG_4       0.897219
AK_SG_5       0.884896
AK_SG_7       0.823304
AK_SG_8       0.833581
AK_SG_8.2     0.845249
AK_SG_9       0.898655
AK_SK_1       0.846300
AK_SK_10      0.858871
AK_SK_11      0.830234
AK_SK_12      0.807735
AK_SK_13      0.842542
AK_SK_14      0.769059
AK_SK_15      0.880364
AK_SK_17      0.814759
AK_SK_18      0.832451
AK_SK_19      0.208518
AK_SK_2       0.857481
AK_SK_24      0.008827
AK_SK_25      0.806957
AK_SK_26      0.819853
AK_SK_27      0.808223
AK_SK_29      0.759318
AK_SK_30      0.678753
AK_SK_32      0.866653
AK_SK_33      0.844319
AK_SK_34      0.806630
AK_SK_35      0.657344
AK_SK_36      0.839912
AK_S

## Save

In [64]:
GPdb.to_csv(data_loc + 'IndianMicrobiome_genomeSetDelta_v1.csv', index=False)
